# Analysis of TCGA RNASeq data using DESeq2
This notebooks uses DESeq2 and R2Py to analyze the RNASeq data sets from TCGA.

In [71]:
import os
import sys
import pandas as pd # version 0.25
import rpy2 # version 3.1.0
from rpy2.robjects import pandas2ri, Formula, r
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter

Note that it is required to install the "DESeq2" package in R. For that run:


if (!requireNamespace("BiocManager", quietly = TRUE))

    install.packages("BiocManager")
    
BiocManager::install("DESeq2")

In [72]:
pandas2ri.activate()
from rpy2.robjects.packages import importr
deseq = importr('DESeq2')

In [73]:
rpy2.__version__

'3.1.0'

In [74]:
pd.__version__

'0.25.1'

Python class defined in https://gist.github.com/wckdouglas/3f8fb27a3d7a1eb24c598aa04f70fb25

In [75]:
to_dataframe = r('function(x) data.frame(x)')

class py_DESeq2:

    def __init__(self, count_matrix, design_matrix, design_formula, gene_column='id'):
        try:
            assert gene_column in count_matrix.columns, 'Wrong gene id column name'
            gene_id = count_matrix[gene_column]
        except AttributeError:
            sys.exit('Wrong Pandas dataframe?')

        self.dds = None
        self.deseq_result = None
        self.resLFC = None
        self.comparison = None
        self.normalized_count_matrix = None
                
        # Remove rows which contain only 0 counts
        count_matrix.set_index('ENSEMBL', inplace=True)
        count_matrix = count_matrix.loc[(count_matrix!=0).any(axis=1)]
        count_matrix.reset_index(inplace=True)
        
        self.gene_column = gene_column
        self.gene_id = count_matrix[self.gene_column]
        
        count_matrix = count_matrix.drop(gene_column,axis=1)
        
        print(f'{count_matrix.shape[1]} | {design_matrix.shape[0]}')
        
        # Load dataframe into R environment
        # Important: Change to r.data() if you use numpys and rpy2 latests versions
        
        # Convert pandas dataframe to r dataframe
        with localconverter(ro.default_converter + pandas2ri.converter):
              count_matrix = ro.conversion.py2rpy(count_matrix)
        
        # Assign columns to NULL
        count_matrix.names = rpy2.rinterface.NULL
        
        self.count_matrix = count_matrix
        
        with localconverter(ro.default_converter + pandas2ri.converter):
              self.design_matrix = ro.conversion.py2rpy(design_matrix)
        
        self.design_formula = Formula(design_formula)


    def run_deseq(self, **kwargs):
        self.dds = deseq.DESeqDataSetFromMatrix(
            countData=self.count_matrix, 
            colData=self.design_matrix,
            design=self.design_formula
        )
        self.dds = deseq.DESeq(self.dds, **kwargs)
        # Previous script had "deseq.counts" instead
        self.normalized_count_matrix = deseq.counts_DESeqDataSet(self.dds, normalized=True)

    def get_deseq_result(self, **kwargs):

        self.comparison = deseq.resultsNames(self.dds)

        self.deseq_result = deseq.results(self.dds, **kwargs)
        self.deseq_result = to_dataframe(self.deseq_result)
        
        # Convert r dataframe to pandas dataframe
        with localconverter(ro.default_converter + pandas2ri.converter):
              self.deseq_result = ro.conversion.rpy2py(self.deseq_result) ## back to pandas dataframe
        self.deseq_result[self.gene_column] = self.gene_id.values
        return self.deseq_result

Get and preprocessing data

In [100]:
# Point to the data & results folder
dir_path = os.path.dirname(os.path.realpath('__file__'))
SOURCE = os.path.join(os.path.abspath(os.path.join(dir_path, os.pardir)))
DATA = os.path.join(SOURCE, 'data', '31316060', 'data', 'rnaseq')
RESULTS = os.path.join(SOURCE, 'data', '31316060', 'results')

counts_path = os.path.join(DATA,'H1299_shBRCA2_d4_counts_matrix.txt')
labels_path = os.path.join(DATA,'H1299_shBRCA2_d4_design_matrix.txt')
results_path = os.path.join(RESULTS,'H1299_shBRCA2_d4_deseq_results.csv')

In [101]:
count_matrix = pd.read_csv(
    counts_path, 
    sep = '\t',
    index_col=0
)

design_matrix = pd.read_csv(
    labels_path,
    sep = '\t',
    index_col=0
)

In [102]:
count_matrix.shape

(38933, 7)

In [103]:
count_matrix.head()

,ENSEMBL,TAR5801A86,TAR5801A54,TAR5801A70,TAR5801A85,TAR5801A69,TAR5801A53
0,DDX11L2,3,2,2,2,2,4
1,WASH7P,185,193,242,253,204,209
2,LOC105376912,1,0,0,0,0,0
3,FAM138A,0,0,0,0,0,0
4,---,0,0,0,1,1,1


In [104]:
design_matrix.shape

(6, 6)

In [105]:
design_matrix.head()

,Name,Rep,CellLine,Genotype,DOX,days
Index,,,,,,
TAR5801A86,H1299_shBRCA2_pos_d4_3,3,H1299,shBRCA2,pos,d4
TAR5801A54,H1299_shBRCA2_pos_d4_1,1,H1299,shBRCA2,pos,d4
TAR5801A70,H1299_shBRCA2_pos_d4_2,2,H1299,shBRCA2,pos,d4
TAR5801A85,H1299_shBRCA2_neg_d4_3,3,H1299,shBRCA2,neg,d4
TAR5801A69,H1299_shBRCA2_neg_d4_2,2,H1299,shBRCA2,neg,d4


Create object

In [106]:
# dds = deseq.DESeqDataSetFromMatrix(countData=r_lihc_count_df, colData=r_lihc_label_df, design=design_formula)
deseq2_exp = py_DESeq2(
    count_matrix=count_matrix,
    design_matrix=design_matrix,
    design_formula='~ DOX',
    gene_column='ENSEMBL'
)

6 | 6


In [107]:
deseq2_exp.run_deseq()

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing



In [108]:
results = deseq2_exp.get_deseq_result()

In [109]:
results.loc['573', :]

baseMean             1777.7
log2FoldChange    -0.084021
lfcSE             0.0869107
stat              -0.966751
pvalue             0.333669
padj               0.583548
ENSEMBL               SNHG3
Name: 573, dtype: object

In [110]:
results.to_csv(results_path)